In [ ]:
from env.balancebot_env import BalancebotEnv
from stable_baselines import PPO2
from stable_baselines.common.policies import ActorCriticPolicy, FeedForwardPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines.common.vec_env import SubprocVecEnv
from stable_baselines.bench import Monitor
from stable_baselines.a2c.utils import linear

import os
import time
import torch
import tensorflow as tf
import numpy as np

In [ ]:
from env.balancebot import BalanceBot
from env.balancebot_env import BalancebotEnv
import numpy as np
import pybullet as p
import os
p.connect(p.DIRECT)
random_init_angle = [np.random.uniform(-0.05, 0.05),
                        np.random.uniform(-0.01, 0.01),
                        np.random.uniform(-0.05, 0.05)]

random_init_orient = p.getQuaternionFromEuler(random_init_angle)
urdf_file = os.path.join("/Users/kazami/workspace/pybullet-example/env/", "urdfs/balancebot_simple.urdf")
balancebot_id = p.loadURDF(urdf_file,
                            basePosition=[0, 0, 0.01],
                            baseOrientation=random_init_orient)
p.getDynamicsInfo(0, 1)
env = BalancebotEnv(render=False)
env.reset()

b = env.balancebot
print(b.getState(-1))
b.changeBaseDynamics()
print(b.getState(-1))
print(b.getState(0))
b.changeRightWheelDynamics()
print(b.getState(0))
print(b.getState(1))
b.changeLeftWheelDynamics()
print(b.getState(1))

In [ ]:
from env.balancebot import BalanceBot
import pybullet as p
import os
import numpy as np
p.connect(p.GUI)
random_init_angle = [np.random.uniform(-0.05, 0.05),
                        np.random.uniform(-0.01, 0.01),
                        np.random.uniform(-0.05, 0.05)]

random_init_orient = p.getQuaternionFromEuler(random_init_angle)
urdf_file = os.path.join("/Users/kazami/workspace/pybullet-example/env/", "balancebot_simple.urdf")
balancebot_id = p.loadURDF(urdf_file,
                            basePosition=[0, 0, 0.01],
                            baseOrientation=random_init_orient)

while (1):
    p.stepSimulation()

In [ ]:
from env.balancebot import BalanceBot
from env.balancebot_env import BalancebotEnv
import numpy as np

env = BalancebotEnv(render=False)
env.reset()
obs = np.array([[1.0, 2.0, 3.0, 4.0, 5.0, 2.0, 3.0, 1.0]], dtype=np.float32)
cur_obs = np.tile(obs, [5, 1])
cur_rwds = np.zeros(5)
random_action_sequences = np.random.uniform(-1, 1, size=[5, 5, 1])
a = env.mb_reward(cur_obs[0], random_action_sequences[:,0,:])
print(a)

In [ ]:
log_dir = "model/gym/{}".format(int(time.time()))
os.makedirs(log_dir, exist_ok=True)

In [ ]:
import sys
print(sys.path)
import numpy



In [ ]:
# Create the environment
def make_env(rank):
    def _init():
        env = BalancebotEnv(render=False)
        env = Monitor(env, os.path.join(log_dir, str(rank)))
        return env
    return _init

num_cpu = 16
env = SubprocVecEnv([make_env(rank=i) for i in range(num_cpu)])


In [ ]:
def const_linear(input_tensor, scope, np_w, np_b):
    in_shape = input_tensor.get_shape().as_list()
    n_input = in_shape[1]
    assert n_input == np_w.shape[0], \
        f"incompatible weight shape: {in_shape} and {np_w.shape}"
    n_output = np_w.shape[1]
    assert n_output == np_b.shape[0], \
        f'incompatible bias shape: {np_w.shape} and {np_b.shape}'
    with tf.variable_scope(scope):
        tf_w = tf.constant(np_w, name='w', dtype=input_tensor.dtype)
        tf_b = tf.constant(np_b, name='b', dtype=input_tensor.dtype)
        return tf.matmul(input_tensor, tf_w) + tf_b

In [ ]:
# Create the RL Agwnt
class CustomPolicy2(ActorCriticPolicy):
    np_pi_w = None
    np_pi_b = None
    np_vf_w = None
    np_vf_b = None
    var_layers = [16]
    
    def __init__(self, sess, ob_space, ac_space, n_env, n_steps, n_batch, reuse=False):
        super(CustomPolicy, self).__init__(sess, ob_space, 
                                           ac_space, n_env, n_steps, 
                                           n_batch, n_lstm=256,
                                           reuse=reuse, scale=False)
        if any([self.np_pi_w is None,
                self.np_pi_b is None,
                self.np_vf_w is None,
                self.np_vf_b is None]):
            raise ValueError('Run CustomPolicy.set_weights first before using this CustomPolicy')
        with tf.variable_scope('model', reuse=reuse):
            activ = tf.tanh
            processed_x = tf.layers.flatten(self.processed_x)
            pi_h = activ(const_linear(processed_x, 'pi_fc0', self.np_pi_w, self.np_pi_b))
            vf_h = activ(const_linear(processed_x, 'vf_fc0', self.np_vf_w, self.np_vf_b))
            for i, layer_size in enumerate(self.var_layers, 1):
                pi_h = activ(linear(pi_h, 'pi_fc' + str(i), n_hidden=layer_size, init_scale=np.sqrt(2)))
                vf_h = activ(linear(vf_h, 'vf_fc' + str(i), n_hidden=layer_size, init_scale=np.sqrt(2)))
            value_fn = linear(vf_h, 'vf', 1)
            pi_latent = pi_h
            vf_latent = vf_h

            self.proba_distribution, self.policy, self.q_value = \
                self.pdtype.proba_distribution_from_latent(pi_latent, vf_latent, init_scale=0.01)

        self.value_fn = value_fn
        self.initial_state = None
        self._setup_init()
    
    def step(self, obs, state=None, mask=None, deterministic=False):
        if deterministic:
            action, value, neglogp = self.sess.run([self.deterministic_action, self._value, self.neglogp],
                                                   {self.obs_ph: obs})
        else:
            action, value, neglogp = self.sess.run([self.action, self._value, self.neglogp],
                                                   {self.obs_ph: obs})
        return action, value, self.initial_state, neglogp

    def proba_step(self, obs, state=None, mask=None):
        return self.sess.run(self.policy_proba, {self.obs_ph: obs})

    def value(self, obs, state=None, mask=None):
        return self.sess.run(self._value, {self.obs_ph: obs})
    
    @classmethod
    def custom_set_weights(cls, np_pi_w, np_pi_b, np_vf_w, np_vf_b, var_layers=None):
        cls.np_pi_w = np_pi_w
        cls.np_pi_b = np_pi_b
        cls.np_vf_w = np_vf_w
        cls.np_vf_b = np_vf_b
        if var_layers:
            cls.var_layers = var_layers

In [ ]:
# put your pytorch numpy weights here
CustomPolicy2.custom_set_weights(np.random.rand(4, 32), np.random.rand(32),
                                np.random.rand(4, 32), np.random.rand(32))

In [ ]:
class CustomPolicy(FeedForwardPolicy):
    def __init__(self, *args, **kwargs):
        super(CustomPolicy, self).__init__(*args, **kwargs,
                                           layers=[32, 8],
                                           feature_extraction="mlp")

In [ ]:
model = PPO2(CustomPolicy, env, verbose=1, tensorboard_log=log_dir+"/tensorboard")

In [ ]:
observation_dim = env.observation_space.shape[0]

In [ ]:
weight_encoder = torch.rand(observation_dim, 32,requires_grad=False).numpy()

In [ ]:
weight_encoder.shape

## How do I put the weight of encoder into model ?
## I want to put the weight into 'pi_fc0' and 'vf_fc0' and set them to untrainable
 

![Network](assets/network.png)

In [ ]:
# Train and Save the agent
model.learn(total_timesteps=2e7, tb_log_name="PPO2")
model.save("ppo_save")

In [ ]:
from tensorflow.python.framework import graph_util
checkpoint = tf.train.get_checkpoint_state("model/20200711_32_8_3/")
checkpoint.model_checkpoint_path
meta_file_path = checkpoint.model_checkpoint_path+".meta"


In [ ]:
restore_graph = tf.Graph()
with restore_graph.as_default():
    saver_new = tf.train.import_meta_graph(meta_file_path, clear_devices=True)
restore_graph_def = restore_graph.as_graph_def()

with tf.Session(graph=restore_graph) as sess:
    saver_new.restore(sess, checkpoint.model_checkpoint_path)
    out_graph_def = graph_util.convert_variables_to_constants(sess,
                                                              restore_graph_def,
                                                              ["output/add"])
    summaryWriter = tf.summary.FileWriter('log/', out_graph_def)

for a in out_graph_def.node:
        #if a.op == "Const":
        #    print(a.name)
        #    print(type(a.attr['value']))
        #    print(tf.make_ndarray(a.attr['value'].tensor))
        print(a)

In [ ]:
with tf.gfile.GFile("model/result.pb", "wb") as fid:
    fid.write(out_graph_def.SerializeToString())

In [ ]:

new_graph = tf.Graph()
new_graph_def = new_graph.as_graph_def()

with tf.gfile.GFile("model/result.pb", "rb") as rf:
    new_graph_def.ParseFromString(rf.read())
with new_graph.as_default():
    tf.import_graph_def(new_graph_def, name="")
new_graph_def
        


In [ ]:
pi_fc0_w_np = ''
pi_fc0_b_np = ''
pi_fc1_w_np = ''
pi_fc1_b_np = ''
pi_w_np = ''
pi_b_np = ''
logstd_np = ''
for a in new_graph_def.node:
    if a.name == "model/pi_fc0/w":
        print(tf.make_ndarray(a.attr['value'].tensor).shape)
        pi_fc0_w = tf.make_ndarray(a.attr['value'].tensor)
        pi_fc0_w = pi_fc0_w.flatten()
        for x in pi_fc0_w:
            pi_fc0_w_np += str(x) + ", "
    if a.name == "model/pi_fc0/b":
        print(tf.make_ndarray(a.attr['value'].tensor).shape)
        pi_fc0_b = tf.make_ndarray(a.attr['value'].tensor).flatten()
        for el in pi_fc0_b:
            pi_fc0_b_np += str(el) + ", "
    if a.name == "model/pi_fc1/w":
        pi_fc1_w = tf.make_ndarray(a.attr['value'].tensor).flatten()
        for el in pi_fc1_w:
            pi_fc1_w_np += str(el) + ", "
    if a.name == "model/pi_fc1/b":
        pi_fc1_b = tf.make_ndarray(a.attr['value'].tensor).flatten()
        for el in pi_fc1_b:
            pi_fc1_b_np += str(el) + ", "
    if a.name == "model/pi/w":
        pi_w = tf.make_ndarray(a.attr['value'].tensor).flatten()
        for el in pi_w:
            pi_w_np += str(el) + ", "
    if a.name == "model/pi/b":
        pi_b = tf.make_ndarray(a.attr['value'].tensor).flatten()
        for el in pi_b:
            pi_b_np += str(el) + ", "
    if a.name == "model/pi/logstd":
        logstd = tf.make_ndarray(a.attr['value'].tensor).flatten()
        for el in logstd:
            logstd_np += str(el) + ", "
print("first")
print(pi_fc0_w_np)
print("second")
print(pi_fc0_b_np)
print("5")
print(pi_fc1_w_np)
print("6")
print(pi_fc1_b_np)
print("7")
print(pi_w_np)
print(" ")
print(pi_b_np)
print(" ")
print(logstd_np)

In [ ]:
input_tensor = new_graph.get_tensor_by_name("input/Ob:0")
output_tensor = new_graph.get_tensor_by_name("model/Tanh_2:0")
itensor = np.array([[1.0, 2.0, 3.0, 4.0, 5.0, 2.0, 3.0, 1.0]], dtype=np.float32)
with tf.Session(graph=new_graph) as sess:
    tf.global_variables_initializer().run()
    pred = sess.run(output_tensor, 
                    feed_dict={input_tensor: itensor})
    print(pred)

In [ ]:
log_dir+"/tensorboard/"
from tensorflow.compat.v1 import AttrValue as _AttrValue
new_graph = tf.Graph()
new_graph_def = new_graph.as_graph_def()

with tf.gfile.GFile("model/202006200542/result.pb", "rb") as rf:
    new_graph_def.ParseFromString(rf.read())
with new_graph.as_default():
    tf.import_graph_def(new_graph_def, name="")
s = 5
for a in new_graph_def.node:
    if a.name == "model/pi/b":
        s = a.attr['value']

## You can open tensorboard at terminal
## For example:
### tensorboard --logdir log_dir+"/tensorboard"

In [ ]:
# delete trained model to demonstrate loading
del model 

In [ ]:
# Create the evaluation env
env = DummyVecEnv([lambda: BalancebotEnv(render=True)])

In [ ]:
# Load the trained agent
model = PPO2.load("ppo_saveg", env=env, policy=CustomPolicy)

In [ ]:

# Enjoy trained agent

for ep in range(1):
    obs = env.reset()
    dones = False
    while not dones:
        action, _states = model.predict(obs)
        obs, rewards, dones, info = env.step(action)

In [ ]:
a = 